In [1]:
import warnings 
warnings.filterwarnings('ignore')
import shutil
import os 
from skimage import io
import numpy as np
import matplotlib.pyplot as plt 
import random
from PIL import Image
from torch.autograd import Variable 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
import copy
import gc


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torchvision

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Функция для обучения 

In [4]:

def train( model, criterion,  optimizer, train_dataloader, val_dataloader, num_epochs ):
    torch.cuda.empty_cache()
    gc.collect()
    torch.backends.cuda.cufft_plan_cache.clear()
    
    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)
    
    best_metric = 0
    best_accuracy_test = 0
    best_accuracy_train = 0

    loss_train_hist = []
    loss_val_hist = []
    accuracy_train_hist = []
    accuracy_val_hist = []

    for epoch in range(num_epochs):
        model.train()
        for i in train_dataloader:
            
            
            
        
            lbl = Variable(torch.nn.functional.one_hot(i['lbl'],num_classes=2).to(device)) 
            data = Variable(i['data'].to(device))

            
            outputs = model.forward(data) #forward pass
            optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
            # obtain the loss functio
        
        
        
            loss = criterion(outputs, lbl.float())


        
        
 
            loss.backward() #calculates the loss of the loss function
 
            optimizer.step() #improve from loss, i.e backprop
    
    
    
        model.eval()
        preds = []
        lbls = []
        for i in val_dataloader:
            
            
            
        
            lbl = list(np.array(i['lbl']))
            lbls = lbls + lbl
    
            outputs = list(np.array(model.forward(Variable(i['data'].to(device))).argmax(1).cpu())) #forward pass
            preds = preds + outputs
        
        
        
    
     
        loss = float(criterion(torch.nn.functional.one_hot(torch.tensor(lbls),num_classes=2).float(),
                           torch.nn.functional.one_hot(torch.tensor(preds), num_classes=2).float()))
        test_acc = accuracy_score(lbls, preds)

        loss_val_hist.append(loss)
        accuracy_val_hist.append(test_acc)
    
        
    
        #print(f'Точность на валедации на эпохе {epoch+1} = {acc}')
        #print(f'Loss на валедации на эпохе {epoch+1} = {loss}')
    
        if test_acc > best_accuracy_test:
        
            best_model_wts = copy.deepcopy(model.state_dict())
            best_accuracy_test = test_acc
    
        preds = []
        lbls = []
        for i in train_dataloader:
        
        
            
            lbl = list(np.array(i['lbl']))
            lbls = lbls + lbl
    
            outputs = list(np.array(model.forward(Variable(i['data'].to(device))).argmax(1).cpu())) #forward pass
            preds = preds + outputs
        
        
        
    
     
        loss = float(criterion(torch.nn.functional.one_hot(torch.tensor(lbls),num_classes=2).float(),
                           torch.nn.functional.one_hot(torch.tensor(preds), num_classes=2).float()))
    
        train_acc = accuracy_score(lbls, preds)
    
        loss_train_hist.append(loss)
        accuracy_train_hist.append(train_acc)

        if train_acc>best_accuracy_train:

          best_accuracy_train = train_acc


        if test_acc == best_accuracy_test:

          if  train_acc>=best_accuracy_train:

            best_model_wts = copy.deepcopy(model.state_dict())


        #print(f'Точность на трейне на эпохе {epoch+1} = {acc}')
        #print(f'Loss на трейне на эпохе {epoch+1} = {loss}')

    return best_model_wts


### Метрики

In [5]:
def get_metrics(lbl, pred):

  return [round(accuracy_score(lbl, pred),3), round(precision_score(lbl, pred),3), round(recall_score(lbl, pred),3), round(f1_score(lbl, pred),3), round(roc_auc_score(lbl, pred),3)]
  

### Финальный тест модели 

In [6]:
def final_test_of_model(model, train_dataloader, val_dataloader):


  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model.to(device)

  preds = []
  lbls = []
  model.eval()
  for i in train_dataloader:


    lbl = list(np.array(i['lbl']))
    lbls = lbls + lbl
    
    outputs = list(np.array(model.forward(Variable(i['data'].to(device))).argmax(1).cpu())) #forward pass
    preds = preds + outputs
        
        
        
    
     
    
    
  train_metrics = get_metrics(lbls, preds)


  preds = []
  lbls = []
  for i in val_dataloader:


    lbl = list(np.array(i['lbl']))
    lbls = lbls + lbl
    
    outputs = list(np.array(model.forward(Variable(i['data'].to(device))).argmax(1).cpu())) #forward pass
    preds = preds + outputs


  test_metrics = get_metrics(lbls, preds)

  return train_metrics, test_metrics

  

  


    
        

### Запись результатов в файл и копирование файла на гугл диск 

In [7]:
def write_and_copy_results(train_metrics, test_metrics, extract_info, model_name):

  file = open(f"/content/results/{extract_info}_{model_name}.txt", "w")


  file.write(f" accuracy_score on train data = {train_metrics[0]} \n")
  file.write(f" precision_score on train data = {train_metrics[1]} \n")
  file.write(f" recall_score on train data = {train_metrics[2]} \n")
  file.write(f" f1_score on train data = {train_metrics[3]} \n")
  file.write(f" roc_auc_score on train data = {train_metrics[4]} \n\n")


  file.write(f" accuracy_score on test data = {test_metrics[0]} \n")
  file.write(f" precision_score on test data = {test_metrics[1]} \n")
  file.write(f" recall_score on test data = {test_metrics[2]} \n")
  file.write(f" f1_score on test data = {test_metrics[3]} \n")
  file.write(f" roc_auc_score on test data = {test_metrics[4]} \n\n")
  

  file.close()


  shutil.copyfile(f"/content/results/{extract_info}_{model_name}.txt",f"/content/drive/MyDrive/CNN_LSTM/results/{extract_info}_{model_name}.txt")


### Bi-lstm

In [8]:
class Bi_LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(Bi_LSTM, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True, bidirectional =True) #lstm
        self.fc_1 =  nn.Linear(self.hidden_size*2*self.num_layers, self.hidden_size*2*self.num_layers)
        

        #self.fc_1 =  nn.Linear(self.hidden_size*2, 1024) #fully connected 1
        self.fc_out = nn.Linear( self.hidden_size*2*self.num_layers,num_classes) #fully connected last layer

        self.relu = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.5)
        self.drop2 = nn.Dropout(p=0.1)

        self.softmax = nn.Softmax()
    
    def forward(self,x):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        h_0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        
        hn = hn.view(-1, self.hidden_size*2*self.num_layers) #reshaping the data for Dense layer next
        out = self.relu(hn)
        
        #out=self.relu(output[:,-1,:])
        out = self.drop2(out)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.drop1(out)
        
        out = self.fc_out(out) #Final Output
        return out

In [9]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True, bidirectional =False) #lstm
        self.fc_1 =  nn.Linear(self.hidden_size*self.num_layers, self.hidden_size*2*self.num_layers)
        

        #self.fc_1 =  nn.Linear(self.hidden_size*2, 1024) #fully connected 1
        self.fc_out = nn.Linear( self.hidden_size*2*self.num_layers,num_classes) #fully connected last layer

        self.relu = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.5)
        self.drop2 = nn.Dropout(p=0.1)

        self.softmax = nn.Softmax()
    
    def forward(self,x):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        
        hn = hn.view(-1, self.hidden_size*self.num_layers) #reshaping the data for Dense layer next
        out = self.relu(hn)
        
        #out=self.relu(output[:,-1,:])
        out = self.drop2(out)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.drop1(out)
        
        out = self.fc_out(out) #Final Output
        return out

### Поиск параметров 

In [10]:
def search_params(dict_of_datasets):


  
  
  for i in list(dict_of_datasets.keys()):


    print(i)
    train_dataloader_extracted = DataLoader(Make_extracted_Dataset({'data':torch.load(dict_of_datasets[i][0]),'lbl':torch.load(dict_of_datasets[i][1])}),batch_size=8)
    test_dataloader_extracted = DataLoader(Make_extracted_Dataset({'data':torch.load(dict_of_datasets[i][2]),'lbl':torch.load(dict_of_datasets[i][3])}))

    for k in ['LSTM','Bi_LSTM']:
      
      print(k)
      num_epochs = 1000 
      learning_rate = 0.00001 

      input_size =  dict_of_datasets[i][4]#number of features
      hidden_size = 1000 #number of features in hidden state
      num_layers = dict_of_datasets[i][5] #number of stacked lstm layers

      num_classes = 2


      if k == 'LSTM':

        

        model = LSTM(num_classes, input_size, hidden_size, num_layers, dict_of_datasets[i][5]) #our lstm class 


      else:


        model = Bi_LSTM(num_classes, input_size, hidden_size, num_layers, dict_of_datasets[i][5]) 


      criterion = torch.nn.CrossEntropyLoss()    
      optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate,weight_decay=0.001) 
      number_of_frames=dict_of_datasets[i][5]

      best_model_wts  = train(model, criterion, optimizer, train_dataloader_extracted, test_dataloader_extracted, num_epochs)

      model.load_state_dict(best_model_wts)

      train_metrics, test_metrics = final_test_of_model(model, train_dataloader_extracted, test_dataloader_extracted)


      write_and_copy_results(train_metrics, test_metrics, i, k)








      







      










### Данные

In [11]:
class Make_extracted_Dataset(Dataset):

    def __init__(self, my_dict, ):
        
        self.my_dict = my_dict
       

    def __len__(self):
        
        #Размер сета
        return len(self.my_dict['lbl'])

        
        
    # Получаем 1 набор данных                                                         
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
                idx = idx.tolist()
                
        
        sample = {'data':self.my_dict['data'][idx], 'lbl': self.my_dict['lbl'][idx]}
            
            
       

        

        return sample

In [12]:
def creat_extracted_dataset(extraktor, dataset,size,number_of_frames):
    
    lbls = []
    inputs = []
    for i in dataset:
        
        extracketed_input = [extraktor(i['data'][k].reshape((number_of_frames,3,720,1280)).float())\
                             .detach().numpy() for k in range(len(i['data']))]
        
        extracketed_input = torch.tensor(extracketed_input).reshape((i['data'].shape[0],number_of_frames,size))
        
        
        
        
        
        
        lbls = lbls +list(i['lbl'])
        inputs = inputs + list(np.array(extracketed_input))
    
    return  np.array(inputs),np.array(lbls),
        



In [13]:
import zipfile
# Прописываем путь к файлу с архивом
zip_file = '/content/drive/MyDrive/CNN_LSTM/extracted_datasets.zip'

# Распаковываем архив
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()


### Перебор

In [14]:
os.mkdir('results')

In [15]:
# ключ имя конфигурации - значение: Путь до данных обучающей выборки , путь до меток обучающей выборки , 
                                  # путь до данных тестовой выборки , путь до меток тестовой выборки , 
                                  # размер выхода экстрактора, количество кадров
                                  
dir_with_datasets = '/content/extracted_datasets/'

dict_of_datasets = {
      
      'alexnet_10_frames' : [dir_with_datasets + 'train_alexnet_number_of_frames_10_data.pt',
                             dir_with_datasets + 'train_alexnet_number_of_frames_10_lbl.pt',
                             dir_with_datasets + 'test_alexnet_number_of_frames_10_data.pt',
                             dir_with_datasets + 'test_alexnet_number_of_frames_10_lbl.pt',
                             1000, 10],
                      
      'alexnet_5_frames' : [dir_with_datasets + 'train_alexnet_number_of_frames_5_data.pt',
                             dir_with_datasets + 'train_alexnet_number_of_frames_5_lbl.pt',
                             dir_with_datasets + 'test_alexnet_number_of_frames_5_data.pt',
                             dir_with_datasets + 'test_alexnet_number_of_frames_5_lbl.pt',
                             1000, 5],
                      

      'resnet152_10_frames' : [dir_with_datasets + 'train_resnet152_number_of_frames_10_data.pt',
                             dir_with_datasets + 'train_resnet152_number_of_frames_10_lbl.pt',
                             dir_with_datasets + 'test_resnet152_number_of_frames_10_data.pt',
                             dir_with_datasets + 'test_resnet152_number_of_frames_10_lbl.pt',
                             2048, 10],
                      
      'resnet152_5_frames' : [dir_with_datasets + 'train_resnet152_number_of_frames_5_data.pt',
                             dir_with_datasets + 'train_resnet152_number_of_frames_5_lbl.pt',
                             dir_with_datasets + 'test_resnet152_number_of_frames_5_data.pt',
                             dir_with_datasets + 'test_resnet152_number_of_frames_5_lbl.pt',
                             2048, 5],
                      


      'vgg16_10_frames' : [dir_with_datasets + 'train_vgg16_number_of_frames_10_data.pt',
                             dir_with_datasets + 'train_vgg16_number_of_frames_10_lbl.pt',
                             dir_with_datasets + 'test_vgg16_number_of_frames_10_data.pt',
                             dir_with_datasets + 'test_vgg16_number_of_frames_10_lbl.pt',
                             1000, 10],
                      
      'vgg16_5_frames' : [dir_with_datasets + 'train_vgg16_number_of_frames_5_data.pt',
                             dir_with_datasets + 'train_vgg16_number_of_frames_5_lbl.pt',
                             dir_with_datasets + 'test_vgg16_number_of_frames_5_data.pt',
                             dir_with_datasets + 'test_vgg16_number_of_frames_5_lbl.pt',
                             1000, 5]

                            }


In [16]:
search_params(dict_of_datasets)

alexnet_10_frames
LSTM
Bi_LSTM
alexnet_5_frames
LSTM
Bi_LSTM
resnet152_10_frames
LSTM
Bi_LSTM
resnet152_5_frames
LSTM
Bi_LSTM
vgg16_10_frames
LSTM
Bi_LSTM
vgg16_5_frames
LSTM
Bi_LSTM
